In [1]:
import sqlite3 #importing sqlite3 to perform sql functions on the database
import matplotlib.pyplot as plt #matplotlib.pyplot for creation of interactive plots

In [2]:
con=sqlite3.connect("hospital.db") #establish a connection to hospital.db to query and update data from the database.

In [ ]:
# Creates a new table named "appointments" in the database with the following columns:
# - id: a unique integer primary key to identify each appointment
# - user_id: the id of the user making the appointment, references the "id" column in the "users" table
# - doctor_id: the id of the doctor associated with the appointment, references the "id" column in the "doctors" table
# - date_time: the date and time of the appointment
# The FOREIGN KEY constraints ensure that only valid user and doctor ids can be associated with an appointment

con.execute("""CREATE TABLE appointments (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    doctor_id INTEGER,
    date_time DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(id),
    FOREIGN KEY (doctor_id) REFERENCES doctors(id)
);
""")


In [ ]:
# Creates a new table named "users" in the database with the following columns:
# - id: a unique integer primary key to identify each user
# - name: the user's full name
# - email: the user's email address

con.execute("""CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT
);
""")

In [ ]:
# Creates a new table named "doctors" in the database with the following columns:
# - id: a unique integer primary key to identify each doctor
# - name: the doctor's full name
# - specialty: the doctor's medical specialty
# - hospital_id: the id of the hospital where the doctor works, references the "id" column in the "hospitals" table
# The FOREIGN KEY constraint ensures that only valid hospital ids can be associated with a doctor


con.execute("""CREATE TABLE doctors (
    id INTEGER PRIMARY KEY,
    name TEXT,
    specialty TEXT,
    hospital_id INTEGER,
    FOREIGN KEY (hospital_id) REFERENCES hospitals(id)
);
""")

In [ ]:
# Creates a new table named "hospitals" in the database with the following columns:
# - id: a unique integer primary key to identify each hospital
# - name: the hospital's name
# - location: the hospital's physical location (e.g. address, city, state)
con.execute("""CREATE TABLE hospitals (
    id INTEGER PRIMARY KEY,
    name TEXT,
    location TEXT
);
""")

In [ ]:
con.execute("select * from hospitals").fetchall()

In [ ]:
con.execute("select * from doctors").fetchall()

In [5]:
cur = con.cursor() #create a cursor object to interact with the database

In [ ]:
# Count the number of appointments
# Execute a SQL query to count the number of rows in the "appointments" table
cur.execute("SELECT COUNT(*) FROM appointments")
# Fetch the first result from the query, which is the count of appointments
total_appointments = cur.fetchone()[0]
# Print the total number of appointments
print("Total Appointments:", total_appointments)

In [ ]:
# Count the number of appointments for each user
# Execute a SQL query to count the number of appointments for each user
# The query joins the "users" table with the "appointments" table on the user_id column
# It groups the results by user ID and returns the user ID, name, and count of appointments
cur.execute("""SELECT users.id, users.name, COUNT(appointments.id) AS num_appointments
               FROM users
               LEFT JOIN appointments ON users.id = appointments.user_id
               GROUP BY users.id""")
# Fetch all the results from the query, which is a list of tuples containing user info and appointment count
user_appointments = cur.fetchall()
# Print the header for the user appointment list
print("\nUser Appointments:")
# Iterate over the results and print each user's appointment count
for user in user_appointments:
    print(user)  # Each user is a tuple containing (id, name, num_appointments)

In [ ]:
# Count the number of appointments for each doctor
# Execute a SQL query to count the number of appointments for each doctor
# The query joins the "doctors" table with the "appointments" table on the doctor_id column
# It groups the results by doctor ID and returns the doctor ID, name, and count of appointments
cur.execute("""SELECT doctors.id, doctors.name, COUNT(appointments.id) AS num_appointments
               FROM doctors
               LEFT JOIN appointments ON doctors.id = appointments.doctor_id
               GROUP BY doctors.id""")
# Fetch all the results from the query, which is a list of tuples containing doctor info and appointment count
doctor_appointments = cur.fetchall()
# Print the header for the doctor appointment list
print("\nDoctor Appointments:")
# Iterate over the results and print each doctor's appointment count
for doctor in doctor_appointments:
    print(doctor)  # Each doctor is a tuple containing (id, name, num_appointments)

In [ ]:
# Count the number of appointments for each hospital
# Execute a SQL query to count the number of appointments for each hospital
# The query joins the "hospitals" table with the "doctors" table on the hospital_id column
# It then joins the result with the "appointments" table on the doctor_id column
# It groups the results by hospital ID and returns the hospital ID, name, and count of appointments
cur.execute("""SELECT hospitals.id, hospitals.name, COUNT(appointments.id) AS num_appointments
               FROM hospitals
               LEFT JOIN doctors ON hospitals.id = doctors.hospital_id
               LEFT JOIN appointments ON doctors.id = appointments.doctor_id
               GROUP BY hospitals.id""")
# Fetch all the results from the query, which is a list of tuples containing hospital info and appointment count
hospital_appointments = cur.fetchall()
# Print the header for the hospital appointment list
print("\nHospital Appointments:")
# Iterate over the results and print each hospital's appointment count
for hospital in hospital_appointments:
    print(hospital)  # Each hospital is a tuple containing (id, name, num_appointments)

In [10]:
# Find the busiest day/time for appointments (assuming date_time is in DATETIME format)
# Execute a SQL query to extract the busiest day/time for appointments
# The query uses the strftime function to format the date_time column as 'YYYY-MM-DD HH:MM'
# It groups the results by the formatted appointment time and counts the number of appointments for each time
# The results are then ordered in descending order by the number of appointments and limited to the top 1 row
cur.execute("""SELECT strftime('%Y-%m-%d %H:%M', date_time) AS appointment_time, COUNT(id) AS num_appointments
               FROM appointments
               GROUP BY appointment_time
               ORDER BY num_appointments DESC
               LIMIT 1""")
# Fetch the single result from the query, which is a tuple containing the busiest time and number of appointments
busiest_time = cur.fetchone()
# Print the busiest time for appointments and the corresponding number of appointments
print("\nBusiest Time for Appointments:", busiest_time[0], "- Number of Appointments:", busiest_time[1])


Busiest Time for Appointments: 2024-05-06 22:00 - Number of Appointments: 107


In [ ]:
# Find the most popular specialty
# Execute a SQL query to count the number of doctors for each specialty
# The query groups the results by specialty and counts the number of doctors for each specialty
# The results are then ordered in descending order by the number of doctors and limited to the top 1 row
cur.execute("""SELECT specialty, COUNT(doctors.id) AS num_doctors
               FROM doctors
               GROUP BY specialty
               ORDER BY num_doctors DESC
               LIMIT 1""")
# Fetch the single result from the query, which is a tuple containing the most popular specialty and number of doctors
most_popular_specialty = cur.fetchone()
# Print the most popular specialty and the corresponding number of doctors
print("\nMost Popular Specialty:", most_popular_specialty[0], "- Number of Doctors:", most_popular_specialty[1])

In [ ]:
# Join appointments with users to get user details
# Execute a SQL query to join the appointments table with the users table
# The query selects the appointment ID, date and time, and user name and email
# The join is performed on the user_id column in the appointments table and the id column in the users table
cur.execute("""SELECT appointments.id, appointments.date_time, users.name AS user_name, users.email AS user_email
               FROM appointments
               INNER JOIN users ON appointments.user_id = users.id""")
# Fetch all the results from the query, which is a list of tuples containing appointment and user details
appointments_with_users = cur.fetchall()
# Print the header for the appointment list with user details
print("Appointments with User Details:")
# Iterate over the results and print each appointment with user details
for appointment in appointments_with_users:
    print(appointment)  # Each appointment is a tuple containing (id, date_time, user_name, user_email)

In [ ]:
# Join appointments with doctors to get doctor details
# Execute a SQL query to join the appointments table with the doctors table
# The query selects the appointment ID, date and time, doctor name, specialty, and hospital name
# The join is performed on the doctor_id column in the appointments table and the id column in the doctors table
# The hospital name is obtained by joining the doctors table with the hospitals table on the hospital_id column
cur.execute("""SELECT appointments.id, appointments.date_time, doctors.name AS doctor_name, doctors.specialty, hospitals.name AS hospital_name
               FROM appointments
               INNER JOIN doctors ON appointments.doctor_id = doctors.id
               INNER JOIN hospitals ON doctors.hospital_id = hospitals.id""")
# Fetch all the results from the query, which is a list of tuples containing appointment and doctor details
appointments_with_doctors = cur.fetchall()
# Print the header for the appointment list with doctor details
print("\nAppointments with Doctor Details:")
# Iterate over the results and print each appointment with doctor details
for appointment in appointments_with_doctors:
    print(appointment) 

In [ ]:
# Join doctors with hospitals to get hospital details
# Execute a SQL query to join the doctors table with the hospitals table
# The query selects the doctor name, specialty, hospital name, and hospital location
# The join is performed on the hospital_id column in the doctors table and the id column in the hospitals table
cur.execute("""SELECT doctors.name AS doctor_name, doctors.specialty, hospitals.name AS hospital_name, hospitals.location
               FROM doctors
               INNER JOIN hospitals ON doctors.hospital_id = hospitals.id""")
# Fetch all the results from the query, which is a list of tuples containing doctor and hospital details
doctors_with_hospitals = cur.fetchall()
# Print the header for the doctor list with hospital details
print("\nDoctors with Hospital Details:")
# Iterate over the results and print each doctor with hospital details
for doctor in doctors_with_hospitals:
    print(doctor)  # Each doctor is a tuple containing (doctor_name, specialty, hospital_name, hospital_location)

In [ ]:
# Analyze appointment distribution by specialty
cur.execute("""SELECT specialty, COUNT(*) AS num_appointments
               FROM appointments
               INNER JOIN doctors ON appointments.doctor_id = doctors.id
               GROUP BY specialty""")
appointment_distribution = cur.fetchall()
specialties, num_appointments = zip(*appointment_distribution)

plt.figure(figsize=(15, 8))
plt.bar(specialties, num_appointments)
plt.title('Appointment Distribution by Specialty')
plt.xlabel('Specialty')
plt.ylabel('Number of Appointments')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze appointment distribution by doctor
cur.execute("""SELECT doctors.name, COUNT(*) AS num_appointments
               FROM appointments
               INNER JOIN doctors ON appointments.doctor_id = doctors.id
               GROUP BY doctors.name""")
appointment_distribution = cur.fetchall()
doctors, num_appointments = zip(*appointment_distribution)

plt.figure(figsize=(15, 8))
plt.bar(doctors, num_appointments, color='lightgreen')
plt.title('Appointment Distribution by Doctor')
plt.xlabel('Doctor')
plt.ylabel('Number of Appointments')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze appointment distribution by hospital location
cur.execute("""SELECT hospitals.location, COUNT(*) AS num_appointments
               FROM appointments
               INNER JOIN doctors ON appointments.doctor_id = doctors.id
               INNER JOIN hospitals ON doctors.hospital_id = hospitals.id
               GROUP BY hospitals.location""")
appointment_distribution = cur.fetchall()
locations, num_appointments = zip(*appointment_distribution)

plt.figure(figsize=(15, 8))
plt.bar(locations, num_appointments, color='lightcoral')
plt.title('Appointment Distribution by Hospital Location')
plt.xlabel('Location')
plt.ylabel('Number of Appointments')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# Analyze user distribution by hospital
cur.execute("""SELECT hospitals.name, COUNT(*) AS num_users
               FROM users
               INNER JOIN doctors ON users.id = doctors.id
               INNER JOIN hospitals ON doctors.hospital_id = hospitals.id
               GROUP BY hospitals.name""")
user_distribution = cur.fetchall()
hospitals, num_users = zip(*user_distribution)

plt.figure(figsize=(15, 8))
plt.bar(hospitals, num_users, color='skyblue')
plt.title('User Distribution by Hospital')
plt.xlabel('Hospital')
plt.ylabel('Number of Users')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# Analyze appointment distribution by date
cur.execute("""SELECT DATE(date_time) AS appointment_date, COUNT(*) AS num_appointments
               FROM appointments
               GROUP BY appointment_date""")
appointment_distribution = cur.fetchall()
dates, num_appointments = zip(*appointment_distribution)

plt.figure(figsize=(15, 8))
plt.plot(dates, num_appointments, marker='o', color='orange')
plt.title('Appointment Distribution by Date')
plt.xlabel('Date')
plt.ylabel('Number of Appointments')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze appointment distribution by hour
cur.execute("""SELECT strftime('%H', date_time) AS appointment_hour, COUNT(*) AS num_appointments
               FROM appointments
               GROUP BY appointment_hour""")
appointment_distribution = cur.fetchall()
hours, num_appointments = zip(*appointment_distribution)

plt.figure(figsize=(15, 8))
plt.bar(hours, num_appointments, color='salmon')
plt.title('Appointment Distribution by Hour')
plt.xlabel('Hour')
plt.ylabel('Number of Appointments')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()
